<a href="https://colab.research.google.com/github/YBenjaminPCondori/Machine-Learning-Gen-AI/blob/main/Occupancy-Monitoring-Conv1D-System-with-Gaussian-Data-Compression/Conv1D_Occupancy_Detection_Research.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Conv1D-Based Occupancy Detection from Multisensor Time-Series Data

## Abstract
This notebook presents an end-to-end Conv1D pipeline for occupancy detection using multisensor environmental time-series data. It is structured for research, reproducibility, and publication-ready reporting.

## 1. Imports and Environment Setup

In [ ]:

import pandas as pd
import numpy as np
import tensorflow as tf

from sklearn.preprocessing import StandardScaler
from sklearn.utils import class_weight
from sklearn.metrics import (
    confusion_matrix, classification_report,
    accuracy_score, precision_score, recall_score,
    f1_score, roc_auc_score, log_loss, roc_curve
)

import matplotlib.pyplot as plt
import seaborn as sns


## 2. Dataset Loading and Preprocessing

In [ ]:

df = pd.read_csv(r"C:/Users/ybenj/Downloads/ALL_CSVS/merged_env_motion.csv")

df.drop_duplicates(inplace=True)
df.columns = df.columns.str.strip().str.lower()
df = df[df['motion'].notna()]

numeric_cols = df.select_dtypes(include=np.number).columns
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].mean())

columns_to_drop = ['smoke', 'device']
if 'co' in df.columns:
    columns_to_drop.append('co')
df.drop(columns=columns_to_drop, errors='ignore', inplace=True)

df['timestamp'] = pd.to_datetime(df['timestamp']).astype('int64') / 1e9

if 'light' in df.columns and df['light'].dtype == bool:
    df['light'] = df['light'].astype(int)

df.drop(columns=['hub', 'home'], inplace=True, errors='ignore')

target = 'motion'
df[target] = df[target].astype(int)
features = [col for col in df.columns if col != target]

scaler = StandardScaler()
df[features] = scaler.fit_transform(df[features])

X = df[features].values.astype(np.float32)
y = df[target].values.astype(np.float32)


## 3. Sliding Window Construction

In [ ]:

def create_windows(data, labels, window_size):
    X_windows, y_labels = [], []
    for i in range(len(data) - window_size):
        X_windows.append(data[i:i+window_size])
        y_labels.append(labels[i+window_size])
    return np.array(X_windows), np.array(y_labels)

TIME_STEPS = 100
BATCH_SIZE = 32

X_windowed, y_windowed = create_windows(X, y, TIME_STEPS)


## 4. TensorFlow Dataset Pipeline

In [ ]:

dataset = tf.data.Dataset.from_tensor_slices((X_windowed, y_windowed))
dataset = dataset.shuffle(1000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

total_batches = len(list(dataset))
train_size = int(0.8 * total_batches)

train_ds = dataset.take(train_size)
test_ds = dataset.skip(train_size)


## 5. Class Imbalance Handling

In [ ]:

class_weights_array = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_windowed.astype(int)),
    y=y_windowed.astype(int)
)

class_weights = dict(enumerate(class_weights_array))


## 6. Conv1D Model Architecture

In [ ]:

model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(TIME_STEPS, len(features))),
    tf.keras.layers.Conv1D(64, 3, activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling1D(2),

    tf.keras.layers.Conv1D(128, 3, activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling1D(2),

    tf.keras.layers.Conv1D(256, 3, activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling1D(2),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)


## 7. Training with Early Stopping

In [ ]:

early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

history = model.fit(
    train_ds,
    epochs=10,
    validation_data=test_ds,
    callbacks=[early_stop],
    class_weight=class_weights
)


## 8. Evaluation

In [ ]:

loss, accuracy = model.evaluate(test_ds)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")


## 9. Metrics and Visualisation

In [ ]:

y_true, y_pred_probs, y_pred_classes = [], [], []

for x_batch, y_batch in test_ds:
    preds = model.predict(x_batch).flatten()
    y_true.extend(y_batch.numpy())
    y_pred_probs.extend(preds)
    y_pred_classes.extend((preds > 0.5).astype(int))

y_true = np.array(y_true).astype(int)
y_pred_probs = np.array(y_pred_probs)
y_pred_classes = np.array(y_pred_classes)

cm = confusion_matrix(y_true, y_pred_classes)
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.title("Confusion Matrix")
plt.show()
